In [13]:
! cp /Users/alex/Documents/bagel-data/allowed_mutants.txt . 

In [23]:
# enter your params

read_dir = '191641_dna_seq_rep_9025003_zip'
wt_pep = '../../../../oligo_design/bglb_model.fa'
allowed_mutants = 'allowed_mutants.txt' 
sample_list = []

In [24]:
# convert all the reads to FASTQ

from Bio import SeqIO
from glob import glob 

for read in glob( '{}/*ab1'.format( read_dir ) ):
    SeqIO.convert( read, 'abi', read.replace( 'ab1', 'fastq' ), 'fastq' )

In [25]:
#! ls 191641_dna_seq_rep_9025003_zip
! ( ls 191641_dna_seq_rep_9025003_zip | cut -d_ -f3 | sort -u ) > sample_list.txt

In [26]:
# read from file 

my_sample_list_file = 'sample_list.txt'

with open( my_sample_list_file ) as fn:
    sample_list = fn.read().split()

# try get a ist of the samples if we don't already have that 

# if not sample_list:
#     my_glob = '{}/*fastq'
#     print 'No sample list, attempting to find one with the glob "{}"'.format( my_glob ) 
#     my_files = glob( my_glob.format( read_dir ) )
#     #my_samples = map( lambda x: x.split( '/' )[1].split( '_' )[2], my_files ) 
#     sample_list = [ x.split( '/' )[1].split( '_' )[2] for x in my_files ] 
#     print sample_list 

In [27]:
print( sample_list )

['A306N-1', 'A306N-1.assembled.fasta', 'A306N-1.assembled.fastq', 'A306N-1.discarded.fastq', 'A306N-1.unassembled.forward.fastq', 'A306N-1.unassembled.reverse.fastq', 'A306N-2', 'A306N-2.assembled.fasta', 'A306N-2.assembled.fastq', 'A306N-2.discarded.fastq', 'A306N-2.unassembled.forward.fastq', 'A306N-2.unassembled.reverse.fastq', 'A306N-3', 'A306N-3.assembled.fasta', 'A306N-3.assembled.fastq', 'A306N-3.discarded.fastq', 'A306N-3.unassembled.forward.fastq', 'A306N-3.unassembled.reverse.fastq', 'E406V-1', 'E406V-1.assembled.fasta', 'E406V-1.assembled.fastq', 'E406V-1.discarded.fastq', 'E406V-1.unassembled.forward.fastq', 'E406V-1.unassembled.reverse.fastq', 'E406V-2', 'E406V-2.assembled.fasta', 'E406V-2.assembled.fastq', 'E406V-2.discarded.fastq', 'E406V-2.unassembled.forward.fastq', 'E406V-2.unassembled.reverse.fastq', 'E406V-3', 'E406V-3.assembled.fasta', 'E406V-3.assembled.fastq', 'E406V-3.discarded.fastq', 'E406V-3.unassembled.forward.fastq', 'E406V-3.unassembled.reverse.fastq', 'H1

In [28]:
# pair and merge the reads 

from subprocess import call, check_output

expect = 0
for sample in sample_list:
    #genscript
    fwd = glob( '{}/*_{}_T7.fastq'.format( read_dir, sample ) ) 
    rev = glob( '{}/*_{}_T7-Ter.fastq'.format( read_dir, sample ) )
    #print( len(fwd), len(rev) )
    
    #operon 
#     fwd = glob( '{}/{}_T7_*.fastq'.format( read_dir, sample ) ) 
#     rev = glob( '{}/{}_T7term_*.fastq'.format( read_dir, sample ) )
#     print len( fwd ), len( rev )

    if len( fwd ) == len( rev ) == 1:
        pear_opts = [ '-e', '2' ] 
        cmd = [ 'pear', '-f', fwd[0], '-r', rev[0], '-o', '{}/{}'.format( read_dir, sample ) ] + pear_opts
        call( cmd )
        expect += 1 

In [29]:
# convert all the assemblies to FASTA

for assembly in glob( '{}/*.assembled.fastq'.format( read_dir ) ):
    SeqIO.convert( assembly, 'fastq', assembly.replace( 'fastq', 'fasta' ), 'fasta' )

In [30]:
# load in allowed mutants for error checking 

with open( allowed_mutants ) as fn:
    allowed_mutants = [ i.strip() for i in fn.readlines() if len(i) > 1 ] 

In [51]:
# diff the assemblies against the wild type protein sequence 

singles = []
for assembly in glob( '{}/*.assembled.fasta'.format( read_dir ) ):
    name = assembly.split( '.' )[0] 
    lines = check_output( [ 'blastx', '-subject', wt_pep, '-query', assembly, '-outfmt', '6 sseq qseq length' ] ).split()
    if lines and lines[0] and lines[1] and lines[2]:        
        z = zip( str( lines[0] ), str( lines[1] ) )
        d = [ '{}{}{}'.format( i, n-1, j ) for n, ( i, j ) in enumerate( z ) if i != j ]
        dstr = '+'.join( d )
        # coded responses 
        if len( d ) == 0: 
            print( name, '\twild type' )
        elif len( d ) == 1:
            if d[0] in allowed_mutants:
                print( name, '\t{}'.format( dstr ) )
                singles.append( ( d[0], name ) )
            else:
                print( name, '\t{}'.format( dstr ) )
                #print( name, '\tbad alignment' )
        elif len( d ) > 1:
            #print name, '\t... ({} mutations)'.format( len( d ) ) 
            print( name, '\t{}'.format( dstr ) )

191641_dna_seq_rep_9025003_zip/A306N-1 	N184K+W185L+R186P+E187*+A188S+F189I+T190Y+A191R+H193P+H194S+I195F+L196S+M197K+C198V+H199S+G200C+I201Y+A202G+S203K+N204H+L205P+H206P+K207*+E208K+G210N+L211V+T212*+G213A+K214E+I215N+G216V+I217A+T218S+L219P+N220S+D225Y+A226T+R230H+A306T
191641_dna_seq_rep_9025003_zip/A306N-2 	A306N
191641_dna_seq_rep_9025003_zip/A306N-3 	A306N
191641_dna_seq_rep_9025003_zip/E406V-1 	A7V+N204Q+L205P+H206A+K207*+E208G+G210S+L211S+T212A+G213R+G216C+I217L+T218S+L219P+N220D+M221Y+H223P+V224C+D225G+A227T+R230P+P231S+V234G+A235S+I244Y+N245H+A249S+Y257F+V262G+G324C+E406V
191641_dna_seq_rep_9025003_zip/E406V-2 	E406V
191641_dna_seq_rep_9025003_zip/E406V-3 	F248V+E406V
191641_dna_seq_rep_9025003_zip/H119Q-1 	wild type
191641_dna_seq_rep_9025003_zip/H119Q-2 	H119Q
191641_dna_seq_rep_9025003_zip/H119Q-3 	wild type
191641_dna_seq_rep_9025003_zip/H178C-1 	H178C
191641_dna_seq_rep_9025003_zip/H178C-2 	T143S+H178C
191641_dna_seq_rep_9025003_zip/H178C-3 	H178C
191641_dna_seq_rep_902

In [52]:
singles

[('A306N', '191641_dna_seq_rep_9025003_zip/A306N-2'),
 ('A306N', '191641_dna_seq_rep_9025003_zip/A306N-3'),
 ('E406V', '191641_dna_seq_rep_9025003_zip/E406V-2'),
 ('H119Q', '191641_dna_seq_rep_9025003_zip/H119Q-2'),
 ('H178C', '191641_dna_seq_rep_9025003_zip/H178C-1'),
 ('H178C', '191641_dna_seq_rep_9025003_zip/H178C-3'),
 ('H178W', '191641_dna_seq_rep_9025003_zip/H178W-2'),
 ('H328N', '191641_dna_seq_rep_9025003_zip/H328N-2'),
 ('I20C', '191641_dna_seq_rep_9025003_zip/I20C-1'),
 ('I20C', '191641_dna_seq_rep_9025003_zip/I20C-3'),
 ('W185L', '191641_dna_seq_rep_9025003_zip/L171N-1'),
 ('M10Q', '191641_dna_seq_rep_9025003_zip/M10Q-1'),
 ('M10Q', '191641_dna_seq_rep_9025003_zip/M10Q-2'),
 ('M10Q', '191641_dna_seq_rep_9025003_zip/M10Q-3'),
 ('N293Y', '191641_dna_seq_rep_9025003_zip/N293Y-1'),
 ('N293Y', '191641_dna_seq_rep_9025003_zip/N293Y-2'),
 ('T175H', '191641_dna_seq_rep_9025003_zip/T175H-2'),
 ('T175H', '191641_dna_seq_rep_9025003_zip/T175H-3'),
 ('W120N', '191641_dna_seq_rep_9025003

In [53]:
dict( singles ) 

{'A306N': '191641_dna_seq_rep_9025003_zip/A306N-3',
 'E406V': '191641_dna_seq_rep_9025003_zip/E406V-2',
 'H119Q': '191641_dna_seq_rep_9025003_zip/H119Q-2',
 'H178C': '191641_dna_seq_rep_9025003_zip/H178C-3',
 'H178W': '191641_dna_seq_rep_9025003_zip/H178W-2',
 'H328N': '191641_dna_seq_rep_9025003_zip/H328N-2',
 'I20C': '191641_dna_seq_rep_9025003_zip/I20C-3',
 'M10Q': '191641_dna_seq_rep_9025003_zip/M10Q-3',
 'N293Y': '191641_dna_seq_rep_9025003_zip/N293Y-2',
 'P41L': '191641_dna_seq_rep_9025003_zip/Y166K-2',
 'T175H': '191641_dna_seq_rep_9025003_zip/T175H-3',
 'W120N': '191641_dna_seq_rep_9025003_zip/W120N-3',
 'W185L': '191641_dna_seq_rep_9025003_zip/L171N-1',
 'Y295W': '191641_dna_seq_rep_9025003_zip/Y295N-3'}

In [55]:
# results 

picks = dict( singles ) 
l = len( picks ) 
print( 'Found {} of {} expected mutants'.format( l, expect ) )

Found 14 of 48 expected mutants
